In [1]:
%matplotlib inline

import pandas as pd
from statistics import mean
import cv2
import numpy as np
from matplotlib import pyplot as plt
def getVideoFrame(name,frnum=45):
    cap= cv2.VideoCapture(name)
    i=0
    while(cap.isOpened()):
        ret, frame = cap.read()
        if i==frnum:
            return frame
        i+=1
    
def frames(name,df):
    for i,row in df.iterrows():
        fn=row['mag_file']
        frame=getVideoFrame(fn)

def getAvgMetrics(pbdf,pbhalfdf):
    if len(pbdf)>0 and len(pbhalfdf)>0:
        result=[mean(pbdf['psnr_mag'].tolist()),
             mean(pbdf['ssim_mag'].tolist()),
             mean(pbdf['ms_ssim_mag'].tolist()),
             mean(pbdf['vmaf_mag'].tolist()),
             mean(pbdf['psnr'].tolist()),
             mean(pbdf['ssim'].tolist()),
             mean(pbdf['ms_ssim'].tolist()),
             mean(pbdf['vmaf'].tolist()),
             mean(pbhalfdf['psnr'].tolist()),
             mean(pbhalfdf['ssim'].tolist()),
             mean(pbhalfdf['ms_ssim'].tolist()),
             mean(pbhalfdf['vmaf'].tolist()),
             mean(pbdf['psnr_best'].tolist()),
             mean(pbdf['ssim_best'].tolist()),
             mean(pbdf['ms_ssim_best'].tolist()),
             mean(pbdf['vmaf_best'].tolist()),
             mean(pbhalfdf['psnr_best'].tolist()),
             mean(pbhalfdf['ssim_best'].tolist()),
             mean(pbhalfdf['ms_ssim_best'].tolist()),
             mean(pbhalfdf['vmaf_best'].tolist()),
            list(set(pbhalfdf['best_scale'].tolist()+pbdf['best_scale'].tolist())),
            list(set(pbhalfdf['best_OFwinsize'].tolist()+pbdf['best_OFwinsize'].tolist())),
            list(set(pbhalfdf['best_bilatfil_size'].tolist()+pbdf['best_bilatfil_size'].tolist()))]            
    return result

def reformat(df):
    cols=df.columns
    mag=cols[0:4]
    sfmd=cols[4:8]
    sfmd2=cols[8:12]
    sfmb=cols[12:16]
    sfmb2=cols[16:20]
    
    formatted_cols=[mag[0].split(' ', 1)[0],
                    sfmd[0].split(' ', 1)[0],
                    sfmd2[0].split(' ', 1)[0],
                    sfmb[0].split(' ', 1)[0],
                    sfmb2[0].split(' ', 1)[0]]
    formatted=pd.DataFrame(columns=formatted_cols)
    for i,row in df.iterrows():
        m=row[mag].values
        strmag='{:.1f}/{:.3f}/{:.3f}/{:.1f}'.format(m[0],m[1],m[2],m[3])
        m=row[sfmd].values
        strsfmd='{:.1f}/{:.3f}/{:.3f}/{:.1f}'.format(m[0],m[1],m[2],m[3])
        m=row[sfmd2].values
        strsfmd2='{:.1f}/{:.3f}/{:.3f}/{:.1f}'.format(m[0],m[1],m[2],m[3])
        m=row[sfmb].values
        strsfmb='{:.1f}/{:.3f}/{:.3f}/{:.1f}'.format(m[0],m[1],m[2],m[3])
        m=row[sfmb2].values
        strsfmb2='{:.1f}/{:.3f}/{:.3f}/{:.1f}'.format(m[0],m[1],m[2],m[3])
        frow=[strmag,strsfmd,strsfmd2,strsfmb,strsfmb2]
        formatted.loc[i] = frow
    return formatted

def plotScores(df,miny=0.4, maxy=0.8, savename=None):
    cols=df.columns
    mag=cols[0:4]
    sfmd=cols[4:8]
    sfmd2=cols[8:12]
    sfmb=cols[12:16]
    sfmb2=cols[16:20]
    
    #barWidth = 0.25
    labels=[mag[0].split(' ', 1)[0],
                    sfmd[0].split(' ', 1)[0],
                    sfmb[0].split(' ', 1)[0],
                    sfmd2[0].split(' ', 1)[0],
                    sfmb2[0].split(' ', 1)[0]]
    
    x = np.arange(len(labels))
    fig, ax = plt.subplots()
    width = 0.095
    n=0
    for i,row in df.iterrows():
        if i=='octave':
            break
        barsi = [score(row[mag].values),
                 score(row[sfmd].values),
                 score(row[sfmb].values),
                 score(row[sfmd2].values),
                 score(row[sfmb2].values)]#row of dataframe
        if n==0:
            r = np.arange(len(barsi))
        else:
            r = [x + width for x in r]
        print(r)
        rects = ax.bar(r, barsi, width, label=i)
        n+=1
        
    # Add xticks on the middle of the group bars
    plt.xlabel('group', fontweight='bold')
    plt.xticks([r + width for r in range(len(barsi))], labels)
    
    ax.set_ylim([miny,maxy])
    major_ticks = np.arange(miny, maxy, 0.05)
    minor_ticks = np.arange(miny, maxy, 0.0125)

    ax.set_yticks(major_ticks)
    ax.set_yticks(minor_ticks, minor=True)
    ax.grid(which='both')
    # Create legend & Show graphic
    plt.legend(loc='best', fancybox=True, framealpha=0.5)

    plt.grid(True,axis='y',alpha=0.5, linestyle='-')
    plt.grid(b=True, which='minor',alpha=0.3, linestyle='--')
    plt.grid(True,axis='x',alpha=0.0, linestyle='')

    plt.rcParams['figure.figsize'] = [15, 6]
    #plt.figure(figsize=(15,8), dpi= 100, facecolor='w', edgecolor='k')
    

    import matplotlib.transforms
    plt.setp( ax.xaxis.get_majorticklabels()) 
    # Create offset transform by 5 points in x direction
    dx = 36/72.; dy = 0/72. 
    offset = matplotlib.transforms.ScaledTranslation(dx, dy, fig.dpi_scale_trans)
    # apply offset transform to all x ticklabels.
    for label in ax.xaxis.get_majorticklabels():
        label.set_transform(label.get_transform() + offset)

    plt.tick_params(
    axis='x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=True) # labels along the bottom edge are off

    if savename is not None:
        plt.savefig(savename)
    plt.show()
def score(input):
    return (input[1]+input[2]+input[3]/100)/3
#plotScores(res)

!pwd
!ls

/home/jonathan/Dropbox/UnB/motionMag/quality_3
backup		     from_PB			    SFM_vs_DMS.png
csvs		     generate_demo_large_motion.py  SFM_vs_PB.png
data_analysis.ipynb  generate_demo_small_motion.py  SynFlowMap_from_ACC.csv
exp_SFM_ACC.py	     latex_table_.txt		    SynFlowMap_from_DMD.csv
exp_SFM_DMD.py	     latex_table.txt		    SynFlowMap_from_DMS.csv
exp_SFM_DMS.py	     SearchResults.ipynb	    SynFlowMap_from_PB.csv
exp_SFM_PB.py	     SFM_gui.py			    temp_csvs
from_ACC	     SFM_large.py		    tmp
from_DMD	     SFM_vs_ACC.png		    videos
from_DMS	     SFM_vs_DMD.png


In [2]:
def diffs(pbdf):
    pbdf['metric']=pbdf['ssim']/3+pbdf['ms_ssim']/3+pbdf['vmaf']/300
    pbdf['metric_best']=pbdf['ssim_best']/3+pbdf['ms_ssim_best']/3+pbdf['vmaf_best']/300
    psnr=pbdf[['psnr','psnr_best']]
    psnr['diff']=psnr['psnr_best']-psnr['psnr']
    ssim=pbdf[['ssim','ssim_best']]
    ssim['diff']=ssim['ssim_best']-ssim['ssim']
    ms_ssim=pbdf[['ms_ssim','ms_ssim_best']]
    ms_ssim['diff']=ms_ssim['ms_ssim_best']-ms_ssim['ms_ssim']
    vmaf=pbdf[['vmaf','vmaf_best']]
    vmaf['diff']=vmaf['vmaf_best']-vmaf['vmaf']
    metric=pbdf[['metric','metric_best']]
    metric['diff']=metric['metric_best']-metric['metric']
    metric

    diffs=pd.DataFrame()
    diffs['metric_diff']=metric['diff']
    diffs['psnr_diff']=psnr['diff']
    diffs['ssim_diff']=ssim['diff']
    diffs['ms_ssim_diff']=ms_ssim['diff']
    diffs['vmaf_diff']=vmaf['diff']
    return diffs

def avgs(pbdf):
    
    pbdf['metric_mag']=pbdf['ssim_mag']/3+pbdf['ms_ssim_mag']/3+pbdf['vmaf_mag']/300
    pbdf['metric']=pbdf['ssim']/3+pbdf['ms_ssim']/3+pbdf['vmaf']/300
    pbdf['metric_best']=pbdf['ssim_best']/3+pbdf['ms_ssim_best']/3+pbdf['vmaf_best']/300

    scores=pbdf[['psnr_mag','ssim_mag','ms_ssim_mag','vmaf_mag','metric_mag','psnr','ssim','ms_ssim','vmaf','metric','psnr_best','ssim_best','ms_ssim_best','vmaf_best','metric_best']]    
    meanT=scores.mean().to_frame().T    
    return meanT, scores.std().T


In [3]:

vnum_plainbg=[]
vnum_texturebg=[]
vnum_moveX=[]
vnum_moveY=[]
vnum_moveXY=[]
for i in range(27):
    if i<9:
        vnum_moveY.append(i)
    elif i<18:
        vnum_moveX.append(i)
    else:
        vnum_moveXY.append(i)
    if i%3==0:
        vnum_plainbg.append(i)
    else:
        vnum_texturebg.append(i)
idx_octave=[]
idx_quarter=[]
for i in range(108):
    if i//27%2==0:
        idx_octave.append(i)
    else:
        idx_quarter.append(i)
pbdf=pd.read_csv('SynFlowMap_from_PB.csv')
pbhalfdf=pbdf[pbdf['alpha_2']==2]
pbdf=pbdf[pbdf['alpha_2']==1]
print(len(pbdf),len(pbhalfdf))

alphas=list(set(pbdf['pb_alpha'].tolist()))
res=pd.DataFrame(columns=['PB PSNR','PB SSIM','PB MS_SSIM','PB VMAF',
                          'SFMD PSNR','SFMD SSIM','SFMD MS_SSIM','SFMD VMAF',
                          'SFMD2 PSNR','SFMD2 SSIM','SFMD2 MS_SSIM','SFMD2 VMAF',
                          'SFMB PSNR','SFMB SSIM','SFMB MS_SSIM','SFMB VMAF',
                          'SFMB2 PSNR','SFMB2 SSIM','SFMB2 MS_SSIM','SFMB2 VMAF',
                          'scales','winsizes','bfilsize'])

res.loc['all videos'] = getAvgMetrics(pbdf,pbhalfdf)

#alphas=alphas.sort()
for a in alphas:
    print('alpha=',a)
    pb1=pbdf[pbdf['pb_alpha']==a]
    pb2=pbhalfdf[pbhalfdf['pb_alpha']==a//2]
    res.loc['alpha='+str(a)]=getAvgMetrics(pb1,pb2)
    
for i in range(2):
    if i==0:
        var='plainBG'
        l=vnum_plainbg
    else:
        var='textureBG'
        l=vnum_texturebg
    print('exp=',var)
    pb1=pbdf[pbdf['video_num'].isin(l)]
    pb2=pbhalfdf[pbhalfdf['video_num'].isin(l)]

    res.loc[var]=getAvgMetrics(pb1,pb2)
    
for i in range(3):
    if i==0:
        var='moveX'
        l=vnum_moveX
    elif i==1:
        var='moveY'
        l=vnum_moveY
    else:
        var='moveXY'
        l=vnum_moveXY    
    print('exp=',var)    
    pb1=pbdf[pbdf['video_num'].isin(l)]
    pb2=pbhalfdf[pbhalfdf['video_num'].isin(l)]
    res.loc[var]=getAvgMetrics(pb1,pb2)

for i in range(2):
    if i==0:
        var='octave'
        l=idx_octave
    else:
        var='quarter octave'
        l=idx_quarter    
    print('exp=',var)    
    pb1=pbdf[pbdf.index.isin(l)]
    pb2=pbhalfdf[pbhalfdf.index.isin(l)]
    res.loc[var]=getAvgMetrics(pb1,pb2)
#plotScores(res,miny=0.4, savename='SFM_vs_PB.png')
res[['SFMD PSNR','SFMD SSIM','SFMB PSNR','SFMB SSIM']]

diffs(pbhalfdf)
pb_scores,_=avgs(pbdf)
#df_pb_scores=pd.DataFrame(pb_scores.T,columns=[['PB']])
pb_scores2,_=avgs(pbhalfdf)
total_scores=pd.DataFrame()
total_scores=pd.concat([total_scores,pb_scores.T],axis=1,names=['PB'])
total_scores=pd.concat([total_scores,pb_scores2.T],axis=1,names=['PB halfmag'])
total_scores_pb=total_scores
total_scores_pb


54 54
alpha= 160
alpha= 40
alpha= 80
exp= plainBG
exp= textureBG
exp= moveX
exp= moveY
exp= moveXY
exp= octave
exp= quarter octave


/home/jonathan/.local/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/home/jonathan/.local/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/home/jonathan/.local/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

,0,0
metric,0.635644,0.647761
metric_best,0.676431,0.684083
metric_mag,0.610760,0.562546
ms_ssim,0.825165,0.836346
ms_ssim_best,0.855874,0.861622
ms_ssim_mag,0.839891,0.770925
psnr,26.027549,26.297787
psnr_best,26.880925,27.047548
psnr_mag,25.898293,24.130790
ssim,0.772500,0.784223


In [4]:
vnum_plainbg=[]
vnum_texturebg=[]
vnum_moveX=[]
vnum_moveY=[]
vnum_moveXY=[]
for i in range(27):
    if i<9:
        vnum_moveY.append(i)
    elif i<18:
        vnum_moveX.append(i)
    else:
        vnum_moveXY.append(i)
    if i%3==0:
        vnum_plainbg.append(i)
    else:
        vnum_texturebg.append(i)
idx_octave=[]
idx_quarter=[]
for i in range(108):
    if i//27%2==0:
        idx_octave.append(i)
    else:
        idx_quarter.append(i)
pbdf=pd.read_csv('SynFlowMap_from_DMS.csv')
pbhalfdf=pbdf[pbdf['alpha_2']==2]
pbdf=pbdf[pbdf['alpha_2']==1]
print(len(pbdf),len(pbhalfdf))

alphas=list(set(pbdf['dms_alpha'].tolist()))
res=pd.DataFrame(columns=['DMS PSNR','DMS SSIM','DMS MS_SSIM','DMS VMAF',
                          'SFMD PSNR','SFMD SSIM','SFMD MS_SSIM','SFMD VMAF',
                          'SFMD2 PSNR','SFMD2 SSIM','SFMD2 MS_SSIM','SFMD2 VMAF',
                          'SFMB PSNR','SFMB SSIM','SFMB MS_SSIM','SFMB VMAF',
                          'SFMB2 PSNR','SFMB2 SSIM','SFMB2 MS_SSIM','SFMB2 VMAF',
                          'scales','winsizes','bfilsize'])
print(pbdf.columns)

res.loc['all videos'] = getAvgMetrics(pbdf,pbhalfdf)

#alphas=alphas.sort()
for a in alphas:
    print('alpha=',a)
    pb1=pbdf[pbdf['dms_alpha']==a]
    pb2=pbhalfdf[pbhalfdf['dms_alpha']==a//2]
    res.loc['alpha='+str(a)]=getAvgMetrics(pb1,pb2)
    
for i in range(2):
    if i==0:
        var='plainBG'
        l=vnum_plainbg
    else:
        var='textureBG'
        l=vnum_texturebg
    pb1=pbdf[pbdf['video_num'].isin(l)]
    pb2=pbhalfdf[pbhalfdf['video_num'].isin(l)]

    res.loc[var]=getAvgMetrics(pb1,pb2)
    
for i in range(3):
    if i==0:
        var='moveX'
        l=vnum_moveX
    elif i==1:
        var='moveY'
        l=vnum_moveY
    else:
        var='moveXY'
        l=vnum_moveXY    
    pb1=pbdf[pbdf['video_num'].isin(l)]
    pb2=pbhalfdf[pbhalfdf['video_num'].isin(l)]
    res.loc[var]=getAvgMetrics(pb1,pb2)
#plotScores(res,miny=0.4,maxy=0.8,savename='SFM_vs_DMS.png')

res
(reformat(res))
res[['SFMD PSNR','SFMD SSIM','SFMB PSNR','SFMB SSIM']]


avgs(pbdf)
dms_scores,_=avgs(pbdf)
dms_scores2,_=avgs(pbhalfdf)
total_scores=pd.concat([total_scores,dms_scores.T],axis=1)
total_scores=pd.concat([total_scores,dms_scores2.T],axis=1)
print(total_scores.round(3).to_latex())
total_scores_dms=total_scores

27 27
Index(['Unnamed: 0', 'alpha_2', 'best_OFwinsize', 'best_bilatfil_size',
       'best_scale', 'dms_alpha', 'dms_settings', 'gt_file', 'mag_file',
       'ms_ssim', 'ms_ssim_best', 'ms_ssim_mag', 'orig_file', 'psnr',
       'psnr_best', 'psnr_mag', 'score', 'score_best', 'score_mag',
       'scorebest_OFwinsize', 'sfm_best_file', 'sfm_default_file', 'ssim',
       'ssim_best', 'ssim_mag', 'status', 'video_num', 'vmaf', 'vmaf_best',
       'vmaf_mag'],
      dtype='object')
alpha= 160
alpha= 40
alpha= 80
\begin{tabular}{lrrrr}
\toprule
{} &       0 &       0 &       0 &       0 \\
\midrule
metric       &   0.636 &   0.648 &   0.513 &   0.508 \\
metric\_best  &   0.676 &   0.684 &   0.517 &   0.511 \\
metric\_mag   &   0.611 &   0.563 &   0.509 &   0.558 \\
ms\_ssim      &   0.825 &   0.836 &   0.707 &   0.698 \\
ms\_ssim\_best &   0.856 &   0.862 &   0.712 &   0.703 \\
ms\_ssim\_mag  &   0.840 &   0.771 &   0.704 &   0.761 \\
psnr         &  26.028 &  26.298 &  22.763 &  22.596 \\
p

/home/jonathan/.local/lib/python3.6/site-packages/ipykernel_launcher.py:82: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

/home/jonathan/.local/lib/python3.6/site-packages/ipykernel_launcher.py:83: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [5]:
vnum_plainbg=[]
vnum_texturebg=[]
vnum_movingbg=[]
vnum_moveX=[]
vnum_moveY=[]
vnum_moveXY=[]
for i in range(27):
    if i<9:
        vnum_moveY.append(i)
    elif i<18:
        vnum_moveX.append(i)
    else:
        vnum_moveXY.append(i)
    if i%3==0:
        vnum_plainbg.append(i)
    elif i%3==1:
        vnum_texturebg.append(i)
    else:
        vnum_movingbg.append(i)
idx_octave=[]
idx_quarter=[]
for i in range(108):
    if i//27%2==0:
        idx_octave.append(i)
    else:
        idx_quarter.append(i)
pbdf=pd.read_csv('SynFlowMap_from_ACC.csv')
pbhalfdf=pbdf[pbdf['alpha_2']==2]
pbdf=pbdf[pbdf['alpha_2']==1]
print(len(pbdf),len(pbhalfdf))

alphas=list(set(pbdf['acc_alpha'].tolist()))
res=pd.DataFrame(columns=['ACC PSNR','ACC SSIM','ACC MS_SSIM','ACC VMAF',
                          'SFMD PSNR','SFMD SSIM','SFMD MS_SSIM','SFMD VMAF',
                          'SFMD2 PSNR','SFMD2 SSIM','SFMD2 MS_SSIM','SFMD2 VMAF',
                          'SFMB PSNR','SFMB SSIM','SFMB MS_SSIM','SFMB VMAF',
                          'SFMB2 PSNR','SFMB2 SSIM','SFMB2 MS_SSIM','SFMB2 VMAF',
                          'scales','winsizes','bfilsize'])
print(pbdf.columns)

res.loc['all videos'] = getAvgMetrics(pbdf,pbhalfdf)

#alphas=alphas.sort()
for a in alphas:
    print('alpha=',a)
    pb1=pbdf[pbdf['acc_alpha']==a]
    pb2=pbhalfdf[pbhalfdf['acc_alpha']==a//2]
    res.loc['alpha='+str(a)]=getAvgMetrics(pb1,pb2)
    
for i in range(3):
    if i==0:
        var='plainBG'
        l=vnum_plainbg
    elif i==1:
        var='textureBG'
        l=vnum_texturebg
    else:
        var='movingBG'
        l=vnum_movingbg
    pb1=pbdf[pbdf['video_num'].isin(l)]
    pb2=pbhalfdf[pbhalfdf['video_num'].isin(l)]

    res.loc[var]=getAvgMetrics(pb1,pb2)
    
for i in range(3):
    if i==0:
        var='moveX'
        l=vnum_moveX
    elif i==1:
        var='moveY'
        l=vnum_moveY
    else:
        var='moveXY'
        l=vnum_moveXY    
    pb1=pbdf[pbdf['video_num'].isin(l)]
    pb2=pbhalfdf[pbhalfdf['video_num'].isin(l)]
    res.loc[var]=getAvgMetrics(pb1,pb2)
#plotScores(res,maxy=0.6,savename='SFM_vs_ACC.png')

diffs(pbdf)
acc_scores,_=avgs(pbdf)
acc_scores2,_=avgs(pbhalfdf)
total_scores=pd.concat([total_scores,acc_scores.T],axis=1)
total_scores=pd.concat([total_scores,acc_scores2.T],axis=1)
total_scores_acc=total_scores

27 27
Index(['Unnamed: 0', 'acc_alpha', 'acc_settings', 'alpha_2', 'best_OFwinsize',
       'best_bilatfil_size', 'best_scale', 'gt_file', 'mag_file', 'ms_ssim',
       'ms_ssim_best', 'ms_ssim_mag', 'orig_file', 'psnr', 'psnr_best',
       'psnr_mag', 'score', 'score_best', 'score_mag', 'scorebest_OFwinsize',
       'sfm_best_file', 'sfm_default_file', 'ssim', 'ssim_best', 'ssim_mag',
       'status', 'video_num', 'vmaf', 'vmaf_best', 'vmaf_mag'],
      dtype='object')
alpha= 16
alpha= 8
alpha= 4


/home/jonathan/.local/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/home/jonathan/.local/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/home/jonathan/.local/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

In [6]:
vnum_plainbg=[]
vnum_texturebg=[]
vnum_movingbg=[]
vnum_moveX=[]
vnum_moveY=[]
vnum_moveXY=[]
for i in range(27):
    if i<9:
        vnum_moveY.append(i)
    elif i<18:
        vnum_moveX.append(i)
    else:
        vnum_moveXY.append(i)
    if i%3==0:
        vnum_plainbg.append(i)
    elif i%3==1:
        vnum_texturebg.append(i)
    else:
        vnum_movingbg.append(i)
idx_octave=[]
idx_quarter=[]
for i in range(108):
    if i//27%2==0:
        idx_octave.append(i)
    else:
        idx_quarter.append(i)
pbdf=pd.read_csv('SynFlowMap_from_DMD.csv')
pbhalfdf=pbdf[pbdf['alpha_2']==2]
pbdf=pbdf[pbdf['alpha_2']==1]
print(len(pbdf),len(pbhalfdf))

alphas=list(set(pbdf['dmd_alpha'].tolist()))
res=pd.DataFrame(columns=['DMD PSNR','DMD SSIM','DMD MS_SSIM','DMD VMAF',
                          'SFMD PSNR','SFMD SSIM','SFMD MS_SSIM','SFMD VMAF',
                          'SFMD2 PSNR','SFMD2 SSIM','SFMD2 MS_SSIM','SFMD2 VMAF',
                          'SFMB PSNR','SFMB SSIM','SFMB MS_SSIM','SFMB VMAF',
                          'SFMB2 PSNR','SFMB2 SSIM','SFMB2 MS_SSIM','SFMB2 VMAF',
                          'scales','winsizes','bfilsize'])
print(pbdf.columns)

res.loc['all videos'] = getAvgMetrics(pbdf,pbhalfdf)

#alphas=alphas.sort()
for a in alphas:
    print('alpha=',a)
    pb1=pbdf[pbdf['dmd_alpha']==a]
    pb2=pbhalfdf[pbhalfdf['dmd_alpha']==a//2]
    res.loc['alpha='+str(a)]=getAvgMetrics(pb1,pb2)
    
for i in range(3):
    if i==0:
        var='plainBG'
        l=vnum_plainbg
    elif i==1:
        var='textureBG'
        l=vnum_texturebg
    else:
        var='movingBG'
        l=vnum_movingbg
    print('exp=',var)
    pb1=pbdf[pbdf['video_num'].isin(l)]
    pb2=pbhalfdf[pbhalfdf['video_num'].isin(l)]

    res.loc[var]=getAvgMetrics(pb1,pb2)
    
for i in range(3):
    if i==0:
        var='moveX'
        l=vnum_moveX
    elif i==1:
        var='moveY'
        l=vnum_moveY
    else:
        var='moveXY'
        l=vnum_moveXY    
    print('exp=',var)    
    pb1=pbdf[pbdf['video_num'].isin(l)]
    pb2=pbhalfdf[pbhalfdf['video_num'].isin(l)]
    res.loc[var]=getAvgMetrics(pb1,pb2)
#plotScores(res,maxy=0.6,savename='SFM_vs_DMD.png')

dmd_scores,_=avgs(pbdf)
dmd_scores2,_=avgs(pbhalfdf)
total_scores=pd.concat([total_scores,dmd_scores.T],axis=1)
total_scores=pd.concat([total_scores,dmd_scores2.T],axis=1)
total_scores_dmd=total_scores
print(total_scores.round(3).to_latex())
sourceFile = open('latex_table.txt', 'w')
print(total_scores.round(3).to_latex(), file = sourceFile)
sourceFile.close()

27 27
Index(['Unnamed: 0', 'alpha_2', 'best_OFwinsize', 'best_bilatfil_size',
       'best_scale', 'dmd_alpha', 'dmd_settings', 'gt_file', 'mag_file',
       'ms_ssim', 'ms_ssim_best', 'ms_ssim_mag', 'orig_file', 'psnr',
       'psnr_best', 'psnr_mag', 'score', 'score_best', 'score_mag',
       'scorebest_OFwinsize', 'sfm_best_file', 'sfm_default_file', 'ssim',
       'ssim_best', 'ssim_mag', 'status', 'video_num', 'vmaf', 'vmaf_best',
       'vmaf_mag'],
      dtype='object')
alpha= 16
alpha= 8
alpha= 4
exp= plainBG
exp= textureBG
exp= movingBG
exp= moveX
exp= moveY
exp= moveXY
\begin{tabular}{lrrrrrrrr}
\toprule
{} &       0 &       0 &       0 &       0 &       0 &       0 &       0 &       0 \\
\midrule
metric       &   0.636 &   0.648 &   0.513 &   0.508 &   0.532 &   0.526 &   0.489 &   0.498 \\
metric\_best  &   0.676 &   0.684 &   0.517 &   0.511 &   0.534 &   0.532 &   0.494 &   0.501 \\
metric\_mag   &   0.611 &   0.563 &   0.509 &   0.558 &   0.518 &   0.536 &   0.482 &   0.

/home/jonathan/.local/lib/python3.6/site-packages/ipykernel_launcher.py:84: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

/home/jonathan/.local/lib/python3.6/site-packages/ipykernel_launcher.py:85: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [7]:
total_scores_pb

,0,0
metric,0.635644,0.647761
metric_best,0.676431,0.684083
metric_mag,0.610760,0.562546
ms_ssim,0.825165,0.836346
ms_ssim_best,0.855874,0.861622
ms_ssim_mag,0.839891,0.770925
psnr,26.027549,26.297787
psnr_best,26.880925,27.047548
psnr_mag,25.898293,24.130790
ssim,0.772500,0.784223


In [8]:
total_scores_dms

,0,0,0,0
metric,0.635644,0.647761,0.513154,0.507637
metric_best,0.676431,0.684083,0.516523,0.510818
metric_mag,0.610760,0.562546,0.508885,0.558290
ms_ssim,0.825165,0.836346,0.706819,0.698435
ms_ssim_best,0.855874,0.861622,0.711637,0.703181
ms_ssim_mag,0.839891,0.770925,0.704328,0.760692
psnr,26.027549,26.297787,22.762773,22.596388
psnr_best,26.880925,27.047548,22.850627,22.681634
psnr_mag,25.898293,24.130790,22.531416,23.975007
ssim,0.772500,0.784223,0.679427,0.675593


In [9]:
total_scores_dmd

,0,0,0,0,0,0,0,0
metric,0.635644,0.647761,0.513154,0.507637,0.531773,0.526124,0.488945,0.498441
metric_best,0.676431,0.684083,0.516523,0.510818,0.534169,0.531853,0.493815,0.501103
metric_mag,0.610760,0.562546,0.508885,0.558290,0.517716,0.536051,0.482190,0.522656
ms_ssim,0.825165,0.836346,0.706819,0.698435,0.714905,0.715443,0.681934,0.690907
ms_ssim_best,0.855874,0.861622,0.711637,0.703181,0.716952,0.717019,0.689109,0.694817
ms_ssim_mag,0.839891,0.770925,0.704328,0.760692,0.716776,0.742325,0.673247,0.726721
psnr,26.027549,26.297787,22.762773,22.596388,22.725221,22.640087,21.970050,22.203454
psnr_best,26.880925,27.047548,22.850627,22.681634,22.777139,22.723716,22.091608,22.289956
psnr_mag,25.898293,24.130790,22.531416,23.975007,23.197529,23.704305,21.707946,22.863300
ssim,0.772500,0.784223,0.679427,0.675593,0.714121,0.712789,0.672295,0.680079


In [10]:
total_scores.columns=['PB','PB halfmag','DMS','DMS halfmag','ACC','ACC halfmag','DMD','DMD halfmag']
total_scores



,PB,PB halfmag,DMS,DMS halfmag,ACC,ACC halfmag,DMD,DMD halfmag
metric,0.635644,0.647761,0.513154,0.507637,0.531773,0.526124,0.488945,0.498441
metric_best,0.676431,0.684083,0.516523,0.510818,0.534169,0.531853,0.493815,0.501103
metric_mag,0.610760,0.562546,0.508885,0.558290,0.517716,0.536051,0.482190,0.522656
ms_ssim,0.825165,0.836346,0.706819,0.698435,0.714905,0.715443,0.681934,0.690907
ms_ssim_best,0.855874,0.861622,0.711637,0.703181,0.716952,0.717019,0.689109,0.694817
ms_ssim_mag,0.839891,0.770925,0.704328,0.760692,0.716776,0.742325,0.673247,0.726721
psnr,26.027549,26.297787,22.762773,22.596388,22.725221,22.640087,21.970050,22.203454
psnr_best,26.880925,27.047548,22.850627,22.681634,22.777139,22.723716,22.091608,22.289956
psnr_mag,25.898293,24.130790,22.531416,23.975007,23.197529,23.704305,21.707946,22.863300
ssim,0.772500,0.784223,0.679427,0.675593,0.714121,0.712789,0.672295,0.680079


In [11]:
table1=total_scores.loc[['psnr_mag','ssim_mag','ms_ssim_mag','vmaf_mag','metric_mag'],['PB','DMS','ACC','DMD']]
table2=total_scores.loc[['psnr','ssim','ms_ssim','vmaf','metric'],['PB','DMS','ACC','DMD']]
table3=total_scores.loc[['psnr_best','ssim_best','ms_ssim_best','vmaf_best','metric_best'],['PB','DMS','ACC','DMD']]
table4=total_scores.loc[['psnr','ssim','ms_ssim','vmaf','metric'],['PB halfmag','DMS halfmag','ACC halfmag','DMD halfmag']]
table4.columns=['PB','DMS','ACC','DMD']
table5=total_scores.loc[['psnr_best','ssim_best','ms_ssim_best','vmaf_best','metric_best'],['PB halfmag','DMS halfmag','ACC halfmag','DMD halfmag']]
table5.columns=['PB','DMS','ACC','DMD']
table=pd.concat([table1,table2,table3,table4,table5])
sourceFile = open('latex_table.txt', 'w')
print(table.round(3).to_latex(), file = sourceFile)

tableT=table.transpose()
sourceFile = open('latex_table_T.txt', 'w')
print(tableT.round(3).to_latex(), file = sourceFile)


In [12]:
print("\\begin{tabular}{lrrrr}\n\\toprule\n{} &      PB &     DMS &     ACC &     DMD \\\\\n\\midrule\npsnr\\_mag     &  25.898 &  22.531 &  23.198 &  21.708 \\\\\nssim\\_mag     &   0.788 &   0.693 &   0.721 &   0.678 \\\\\nms\\_ssim\\_mag  &   0.840 &   0.704 &   0.717 &   0.673 \\\\\nvmaf\\_mag     &  20.424 &  12.919 &  11.565 &   9.570 \\\\\nmetric\\_mag   &   0.611 &   0.509 &   0.518 &   0.482 \\\\\npsnr         &  26.028 &  22.763 &  22.725 &  21.970 \\\\\nssim         &   0.772 &   0.679 &   0.714 &   0.672 \\\\\nms\\_ssim      &   0.825 &   0.707 &   0.715 &   0.682 \\\\\nvmaf         &  30.927 &  15.322 &  16.629 &  11.261 \\\\\nmetric       &   0.636 &   0.513 &   0.532 &   0.489 \\\\\npsnr\\_best    &  26.881 &  22.851 &  22.777 &  22.092 \\\\\nssim\\_best    &   0.802 &   0.683 &   0.716 &   0.676 \\\\\nms\\_ssim\\_best &   0.856 &   0.712 &   0.717 &   0.689 \\\\\nvmaf\\_best    &  37.120 &  15.479 &  16.941 &  11.587 \\\\\nmetric\\_best  &   0.676 &   0.517 &   0.534 &   0.494 \\\\\npsnr         &  26.298 &  22.596 &  22.640 &  22.203 \\\\\nssim         &   0.784 &   0.676 &   0.713 &   0.680 \\\\\nms\\_ssim      &   0.836 &   0.698 &   0.715 &   0.691 \\\\\nvmaf         &  32.271 &  14.888 &  15.014 &  12.434 \\\\\nmetric       &   0.648 &   0.508 &   0.526 &   0.498 \\\\\npsnr\\_best    &  27.048 &  22.682 &  22.724 &  22.290 \\\\\nssim\\_best    &   0.810 &   0.679 &   0.715 &   0.682 \\\\\nms\\_ssim\\_best &   0.862 &   0.703 &   0.717 &   0.695 \\\\\nvmaf\\_best    &  38.091 &  15.072 &  16.327 &  12.681 \\\\\nmetric\\_best  &   0.684 &   0.511 &   0.532 &   0.501 \\\\\n\\bottomrule\n\\end{tabular}\n\n")

\begin{tabular}{lrrrr}
\toprule
{} &      PB &     DMS &     ACC &     DMD \\
\midrule
psnr\_mag     &  25.898 &  22.531 &  23.198 &  21.708 \\
ssim\_mag     &   0.788 &   0.693 &   0.721 &   0.678 \\
ms\_ssim\_mag  &   0.840 &   0.704 &   0.717 &   0.673 \\
vmaf\_mag     &  20.424 &  12.919 &  11.565 &   9.570 \\
metric\_mag   &   0.611 &   0.509 &   0.518 &   0.482 \\
psnr         &  26.028 &  22.763 &  22.725 &  21.970 \\
ssim         &   0.772 &   0.679 &   0.714 &   0.672 \\
ms\_ssim      &   0.825 &   0.707 &   0.715 &   0.682 \\
vmaf         &  30.927 &  15.322 &  16.629 &  11.261 \\
metric       &   0.636 &   0.513 &   0.532 &   0.489 \\
psnr\_best    &  26.881 &  22.851 &  22.777 &  22.092 \\
ssim\_best    &   0.802 &   0.683 &   0.716 &   0.676 \\
ms\_ssim\_best &   0.856 &   0.712 &   0.717 &   0.689 \\
vmaf\_best    &  37.120 &  15.479 &  16.941 &  11.587 \\
metric\_best  &   0.676 &   0.517 &   0.534 &   0.494 \\
psnr         &  26.298 &  22.596 &  22.640 &  22.203 \\
ssim

In [13]:
print(tableT.round(3).to_latex())


\begin{tabular}{lrrrrrrrrrrrrrrrrrrrrrrrrr}
\toprule
{} &  psnr\_mag &  ssim\_mag &  ms\_ssim\_mag &  vmaf\_mag &  metric\_mag &    psnr &   ssim &  ms\_ssim &    vmaf &  metric &  psnr\_best &  ssim\_best &  ms\_ssim\_best &  vmaf\_best &  metric\_best &    psnr &   ssim &  ms\_ssim &    vmaf &  metric &  psnr\_best &  ssim\_best &  ms\_ssim\_best &  vmaf\_best &  metric\_best \\
\midrule
PB  &    25.898 &     0.788 &        0.840 &    20.424 &       0.611 &  26.028 &  0.772 &    0.825 &  30.927 &   0.636 &     26.881 &      0.802 &         0.856 &     37.120 &        0.676 &  26.298 &  0.784 &    0.836 &  32.271 &   0.648 &     27.048 &      0.810 &         0.862 &     38.091 &        0.684 \\
DMS &    22.531 &     0.693 &        0.704 &    12.919 &       0.509 &  22.763 &  0.679 &    0.707 &  15.322 &   0.513 &     22.851 &      0.683 &         0.712 &     15.479 &        0.517 &  22.596 &  0.676 &    0.698 &  14.888 &   0.508 &     22.682 &      0.679 &         0.703 &     15.072 &